In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")

In [2]:
# from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
# from langchain_community.vectorstores.elastic_vector_search import ElasticKnnSearch
# %pip install lark
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers import MultiQueryRetriever, SelfQueryRetriever
from pydantic.v1 import BaseModel, Field  # <-- Uses v1 namespace
from langchain.tools import StructuredTool

In [3]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


In [4]:
from langchain.chains.query_constructor.base import AttributeInfo
metadata_field_info = [
    AttributeInfo(
        name="text",
        description="The Content of news article",
        type="string",
    ),
    AttributeInfo(
        name="vector",
        description="this field is embedding of text field",
        type="vector",
    ),
    AttributeInfo(
        name="title",
        description="The title of the article or document",
        type="string",
    ),
    AttributeInfo(
        name="created_date",
        description="The creation date of the article or document. That field format is 'YYYY-MM-DD' ",
        type="date",
    ),
    AttributeInfo(
        name="portal",
        description="portal of the article or document. It can be daum, naver, etc",
        type="string",
    ),
    AttributeInfo(
        name="media", description="The newspaper that created the article", type="String"
    ),
    AttributeInfo(
        name="url", description="The URL address where the data was posted", type="String"
    ),
    AttributeInfo(
        name="image_url", description="The URL of the image included in the article", type="String"
    ),
]
document_content_description = "this document is news article"

In [5]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
vector_store = ElasticsearchStore(
    embedding=embeddings,
    index_name=ES_INDEX_NAME,
    es_connection=es,
    strategy=DenseVectorStrategy(hybrid=True),
)

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_6765/1668103102.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [6]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
    enable_limit=True
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser


In [8]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator

# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     vector_store,
#     document_content_description,
#     metadata_field_info,
#     structured_query_translator=ElasticsearchTranslator()
# )

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vector_store,    
    structured_query_translator=ElasticsearchTranslator(),
)
retrievers_multi = MultiQueryRetriever.from_llm(
        retriever=retriever,
        llm=llm,
#     document_content_description,
#     metadata_field_info,
    # structured_query_translator=ElasticsearchTranslator()
)


In [10]:
# query_constructor.invoke("삼성전자와 HMB 관련된 2024년 6월 이후 뉴스 관련도가 높은 순서대로 10개를 보여줘")

In [13]:
# result = retrievers_multi.invoke("삼성전자와 HMB 관련된 2024년 6월 이후 뉴스 관련도가 높은 순서대로 10개를 보여줘")
result = retrievers_multi.get_relevant_documents("삼성전자와 HMB 관련된 2024년 6월 이후 뉴스 관련도가 높은 순서대로 10개를 보여줘")
print(len(result))

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 2024년 6월 이후 삼성전자와 HMB에 관한 최신 뉴스 10개를 순위별로 보여주세요.', '2. 삼성전자와 HMB에 대한 2024년 6월 이후의 주요 뉴스 10개를 관련도 높은 순으로 제공해 주세요.', '3. 2024년 6월 이후 삼성전자와 HMB 관련 뉴스 중에서 가장 관련성이 높은 10개를 찾아주세요.']
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being ma

10


/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
for article in result:
    print(article)
    print("")

page_content='5일 한국거래소에 따르면 이날 3.59% 상승 출발한 삼성전자의 주가는 전일 대비 2100원(2.79%) 오른 7만7400원에 거래를 마감했다. 삼성전자는 엔비디아 호재에 힘입어 전날 장 마감후 시간외거래에서 4.12%까지 치솟기도 했다.
업계에 따르면 황 CEO는 전날 열린 기자간담회에서 삼성전자 HBM의 엔비디아 제품 탑재 계획에 대해 "삼성전자와 SK하이닉스, 마이크론과 모두 협력 중이고, 이들 업체에서 모두 제품을 제공받을 것"이라는 입장을 밝혔다.
그는 최근 삼성전자 HBM이 발열 등 문제로 엔비디아의 퀄 테스트를 통과하지 못했다는 추측과 관련해 "어제도 테스트가 진행 중이었고 테스트를 통과하지 못한 게 아니다"고 일축했다.
황 CEO가 삼성전자 HBM의 퀄테스트 실패 루머를 정면 반박한데다 HBM 제품 공급에 대한 긍정적인 답을 내놓으면서 삼성전자의 HMB 엔비디아 납품 가능성이 커질 전망이다.
박유악 키움증권 연구원은 "삼성전자는 엔디비아향 HBM3(4세대)와 HBM3E(5세대) 모두 퀄 테스트가 진행 중에 있다"며 "HBM 뿐만 아니라 9세대 V낸드 양산에도 나서며, 메모리 기술 리더십 탈환에 나설 것"이라고 전망했다.
한동희 SK증권 연구원은 "AI 수요 강세와 HBM 중요성, 기술 경쟁, 수율 저하, 가격 반등세 등을 감안하면 삼성전자의 내년 HBM 계약은 올해 3분기 중 마무리 될 가능성이 매우 높다"고 판단했다.
한편 삼성전자는 현재 5세대 HBM인 '12단 HBM3E'의 엔비디아 퀄테스트 통과에 주력하고 있다.
☞공감언론 뉴시스 byh@newsis.com' metadata={'id': 630328, 'hash_key': '2f2dc9198216783cf76625f371dcd4da', 'title': '젠슨 황 "HBM 테스트 실패 아냐" 삼성전자, 2.79%↑[핫스탁](종합)', 'created_date': '2024-06-05T16:04:50', 'portal': 'daum', 'media': '뉴시스', 'u

In [16]:
# Geneator

In [17]:
llm.invoke("sora에 대해서 알려줘")

AIMessage(content='"Sora"는 여러 가지 의미를 가질 수 있습니다. 어떤 맥락에서 "Sora"에 대해 알고 싶은지 구체적으로 말씀해 주시면 더 정확한 정보를 제공해 드릴 수 있습니다. 아래는 몇 가지 주요한 "Sora"의 예시입니다:\n\n1. **킹덤 하츠 (Kingdom Hearts) 시리즈**:\n   - "Sora"는 스퀘어 에닉스와 디즈니가 공동 개발한 비디오 게임 시리즈 "킹덤 하츠"의 주인공입니다. 그는 키블레이드라는 무기를 사용하여 어둠의 세력과 싸우며, 다양한 디즈니 세계를 여행합니다.\n\n2. **Sora (소라) - 일본어**:\n   - 일본어로 "Sora" (空)는 "하늘"을 의미합니다. 이는 일상 대화에서 자주 사용되는 단어입니다.\n\n3. **Sora (소라) - 인명**:\n   - "Sora"는 일본에서 흔히 사용되는 이름 중 하나입니다. 남성과 여성 모두에게 사용될 수 있습니다.\n\n4. **Sora (소라) - 만화 및 애니메이션**:\n   - 여러 만화 및 애니메이션 작품에서 "Sora"라는 이름을 가진 캐릭터들이 등장합니다. 예를 들어, "디지몬 어드벤처" 시리즈의 "타케노우치 소라"가 있습니다.\n\n5. **Sora (소라) - 기술 및 소프트웨어**:\n   - "Sora"는 마이크로소프트 리서치에서 개발한 소프트웨어 정의 라디오 플랫폼의 이름이기도 합니다.\n\n어떤 "Sora"에 대해 더 알고 싶은지 구체적으로 말씀해 주시면, 더 자세한 정보를 제공해 드리겠습니다.', response_metadata={'token_usage': {'completion_tokens': 387, 'prompt_tokens': 13, 'total_tokens': 400}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-303bcbd9-